In [29]:
import os

import numpy as np

import pickle as pkl
import gzip

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
data_dir = '/'.join(os.getcwd().split("/")[:-1]) + '/data/'
emb_dir = '/'.join(os.getcwd().split("/")[:-1]) + '/embeddings/'

SPECIAL_SYMBOLS_ID = \
    PAD_ID, OOV_ID, EOS_ID = \
    0, 1, 5
SPECIAL_SYMBOLS_STR = \
    PAD_STR, OOV_STR, EOS_STR = \
    "<pad>", "<unk>", "</s>"
NUM_SPECIAL = 3

def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pkl.load(f)
        return loaded_object

In [20]:
#Embeddings
vi_emb = load_zipped_pickle(emb_dir + "vi_embeddings_100K.p")
vi_emb_np = np.array([v for k,v in vi_emb.items()])
vi_id2word = load_zipped_pickle(emb_dir + "id2word_vi_dic.p")
vi_word2id = load_zipped_pickle(emb_dir + "word2id_vi_dic.p")

#Vietnamese Tokens
vi_train_tokens = load_zipped_pickle(data_dir + "vi-en-tokens/train_vi_tok.p")
vi_train_tokens_num = load_zipped_pickle(data_dir + "vi-en-tokens/train_vi_tok_num.p")

vi_dev_tokens = load_zipped_pickle(data_dir + "vi-en-tokens/dev_vi_tok.p")
vi_dev_tokens_num = load_zipped_pickle(data_dir + "vi-en-tokens/dev_vi_tok_num.p")

vi_test_tokens = load_zipped_pickle(data_dir + "vi-en-tokens/test_vi_tok.p")
vi_test_tokens_num = load_zipped_pickle(data_dir + "vi-en-tokens/test_vi_tok_num.p")

#English tokens
en_train_tokens = load_zipped_pickle(data_dir + "vi-en-tokens/train_en_tok.p")
en_train_tokens_num = load_zipped_pickle(data_dir + "vi-en-tokens/train_en_tok_num.p")

en_dev_tokens = load_zipped_pickle(data_dir + "vi-en-tokens/dev_en_tok.p")
en_dev_tokens_num = load_zipped_pickle(data_dir + "vi-en-tokens/dev_en_tok_num.p")

en_test_tokens = load_zipped_pickle(data_dir + "vi-en-tokens/test_en_tok.p")
en_test_tokens_num = load_zipped_pickle(data_dir + "vi-en-tokens/test_en_tok_num.p")



In [38]:
class HybridEmbeddings(nn.Module):
    def __init__(self, fixed_embeddings, learned_embeddings):
        super(HybridEmbeddings, self).__init__()
        self.fixed_embeddings = fixed_embeddings
        self.num_fixed = self.fixed_embeddings.num_embeddings - 1

        self.learned_embeddings = learned_embeddings
        self.num_learned = self.learned_embeddings.num_embeddings - 1

    @property
    def embedding_dim(self):
        return self.fixed_embeddings.embedding_dim

    def forward(self, ids_tensor):
        fixed_ids = transform_ids(
            ids_tensor,
            start=NUM_SPECIAL,
            end=NUM_SPECIAL + self.num_fixed,
        )
        learned_ids = transform_ids(
            ids_tensor,
            start=NUM_SPECIAL + self.num_fixed,
            end=NUM_SPECIAL + self.num_fixed + self.num_learned,
        )
        embeddings = (
            self.fixed_embeddings(fixed_ids)
            + self.learned_embeddings(learned_ids)
        )
        return embeddings

In [58]:
vi_emb_fixed = vi_emb_np[[i for i in range(2, vi_emb_np.shape[0]) if i != 5],:]

fixed_embeddings = nn.Embedding(
        vi_emb_fixed.shape[0],
        vi_emb_fixed.shape[1],
        padding_idx=0,
    )
fixed_embeddings.weight.data.copy_(
        torch.from_numpy(vi_emb_fixed))
fixed_embeddings.weight.requires_grad = False
learned_embeddings = nn.Embedding(
        NUM_SPECIAL+1,
        vi_emb_fixed.shape[1],
        padding_idx=0,
    )
embeddings = HybridEmbeddings(
        fixed_embeddings=fixed_embeddings,
        learned_embeddings=learned_embeddings,
    )

In [61]:
transform_ids(
            ids_tensor,
            start=NUM_SPECIAL,
            end=NUM_SPECIAL + embeddings.num_fixed,
        )

NameError: name 'ids_tensor' is not defined

In [50]:
def transform_ids(ids_tensor, start, end):
        return (ids_tensor - start + 1) * (
            (ids_tensor >= start) & (ids_tensor < end)
        ).long()

In [64]:
vi_word2id[ '</s>']

5